In [ ]:
!pip install biopython
!apt-get update
!apt-get install dssp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [60.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,545 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 

In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

def calculate_secondary_structure_percentage(pdb_file):
    # Parse the PDB structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("Protein", pdb_file)

    # Extract DSSP data
    model = structure[0]  # Use the first model
    dssp = DSSP(model, pdb_file)

    # Define secondary structure categories
    helix_codes = {'H', 'G', 'I'}  # Alpha-helix, 310-helix, Pi-helix
    sheet_codes = {'E', 'B'}  # Extended strand in beta-sheet
    coil_codes = {'T', 'S', 'C', ' '}  # Turn, Bend, and Coil/Undefined

    # Count residues in each category
    helix_count = 0
    sheet_count = 0
    coil_count = 0

    for residue in dssp:
        ss_code = residue[2]  # Secondary structure code
        if ss_code in helix_codes:
            helix_count += 1
        elif ss_code in sheet_codes:
            sheet_count += 1
        elif ss_code in coil_codes:
            coil_count += 1

    # Calculate total residues
    total_residues = helix_count + sheet_count + coil_count

    # Calculate percentages
    helix_percentage = (helix_count / total_residues) * 100 if total_residues > 0 else 0
    sheet_percentage = (sheet_count / total_residues) * 100 if total_residues > 0 else 0
    coil_percentage = (coil_count / total_residues) * 100 if total_residues > 0 else 0

    # Return percentages
    return {
        "Helix (%)": helix_percentage,
        "Sheet (%)": sheet_percentage,
        "Coil (%)": coil_percentage
    }

TP_list = ["ZP3", "ADA"]

for TP in TP_list:
  pdb_file = f"{TP}.pdb"
  percentages = calculate_secondary_structure_percentage(pdb_file)
  print(percentages)


{'Helix (%)': 73.21428571428571, 'Sheet (%)': 7.142857142857142, 'Coil (%)': 19.642857142857142}
{'Helix (%)': 58.048780487804876, 'Sheet (%)': 15.609756097560975, 'Coil (%)': 26.34146341463415}


#**PDB file formatting**

In order to adapt AlphaFold-derived PDB files, which contain `MODEL 1` as internal headers within file metadata instead of `CRYST 1`, all PDB files must be formatted and re-written.

In [ ]:
import pandas as pd

data = pd.read_csv("BaitTP.csv")


def fix_pdb_header(input_pdb, output_pdb):
    with open(input_pdb, 'r') as infile:
        lines = infile.readlines()

    with open(output_pdb, 'w') as outfile:
        for line in lines:
            if line.startswith("MODEL 1") or line.startswith("REMARK   3  ALN A OFF 0"):
                outfile.write("CRYST1   80.000   80.000   80.000  90.00  90.00  90.00 P 1           1\n")
            else:
                outfile.write(line)

# Example usage
for i in range(len(data)):
    if data["Nature"][i] != "TP":
      input_pdb = f"{data['Acronym'][i]}.pdb"
      output_pdb = f"{data['Acronym'][i]}.pdb"
      fix_pdb_header(input_pdb, output_pdb)
      print("PDB header fixed and saved to:", output_pdb)

def clean_pdb_header(input_pdb, output_pdb):
    with open(input_pdb, 'r') as infile:
        lines = infile.readlines()

    with open(output_pdb, 'w') as outfile:
        write_lines = False  # Flag to identify when to start writing lines
        for line in lines:
            if line.startswith("CRYST1"):
                write_lines = True  # Start writing from 'CRYST1'
            if write_lines:
                outfile.write(line)

# Example usage
for i in range(len(data)):
    if data["Nature"][i] != "TP":
      if data["Acronym"][i] != "Mcr-1" and data["Acronym"][i] != "CytB":
          input_pdb = f"{data['Acronym'][i]}.pdb"
          output_pdb = f"{data['Acronym'][i]}.pdb"
          clean_pdb_header(input_pdb, output_pdb)
          print("PDB file cleaned and saved to:", output_pdb)


PDB header fixed and saved to: Mcr-1.pdb
PDB header fixed and saved to: EptA.pdb
PDB header fixed and saved to: AroC.pdb
PDB header fixed and saved to: F3H5.pdb
PDB header fixed and saved to: CHS.pdb
PDB header fixed and saved to: FPPS.pdb
PDB header fixed and saved to: MatK.pdb
PDB header fixed and saved to: PetA.pdb
PDB header fixed and saved to: PsaB.pdb
PDB header fixed and saved to: MVAK.pdb
PDB header fixed and saved to: DBH.pdb
PDB header fixed and saved to: CytB.pdb
PDB header fixed and saved to: LOS.pdb
PDB header fixed and saved to: CADS.pdb
PDB header fixed and saved to: GEAS.pdb
PDB header fixed and saved to: BDH.pdb
PDB header fixed and saved to: PINS.pdb
PDB header fixed and saved to: BDNF.pdb
PDB header fixed and saved to: CysG.pdb
PDB header fixed and saved to: TNF.pdb
PDB header fixed and saved to: MyrS.pdb
PDB header fixed and saved to: GST2.pdb
PDB header fixed and saved to: Am2.pdb
PDB header fixed and saved to: ANXA1.pdb
PDB header fixed and saved to: SMF-1.pdb
PDB

In [ ]:
data = pd.read_csv("BaitTP.csv")
def fix_pdb_header(input_pdb, output_pdb):
    with open(input_pdb, 'r') as infile:
        lines = infile.readlines()

    with open(output_pdb, 'w') as outfile:
        for line in lines:
            if line.startswith("MODEL"):
                outfile.write("CRYST1   80.000   80.000   80.000  90.00  90.00  90.00 P 1           1\n")
            else:
                outfile.write(line)
for i in range(len(data)):
    if data["Nature"][i] == "TP":
      input_pdb = f"{data['Acronym'][i]}.pdb"
      output_pdb = f"{data['Acronym'][i]}.pdb"
      fix_pdb_header(input_pdb, output_pdb)
      print("PDB header fixed and saved to:", output_pdb)
      output_pdb = f"{data['Acronym'][i]}.pdb"
      clean_pdb_header(input_pdb, output_pdb)
      print("PDB file cleaned and saved to:", output_pdb)

PDB header fixed and saved to: phi29.pdb
PDB file cleaned and saved to: phi29.pdb
PDB header fixed and saved to: GC1.pdb
PDB file cleaned and saved to: GC1.pdb
PDB header fixed and saved to: PRD1.pdb
PDB file cleaned and saved to: PRD1.pdb
PDB header fixed and saved to: Bam35.pdb
PDB file cleaned and saved to: Bam35.pdb
PDB header fixed and saved to: Sato.pdb
PDB file cleaned and saved to: Sato.pdb
PDB header fixed and saved to: hAd2.pdb
PDB file cleaned and saved to: hAd2.pdb
PDB header fixed and saved to: viph1008o.pdb
PDB file cleaned and saved to: viph1008o.pdb
PDB header fixed and saved to: ABV.pdb
PDB file cleaned and saved to: ABV.pdb
PDB header fixed and saved to: His1.pdb
PDB file cleaned and saved to: His1.pdb
PDB header fixed and saved to: DSL-LC02.pdb
PDB file cleaned and saved to: DSL-LC02.pdb
PDB header fixed and saved to: St134.pdb
PDB file cleaned and saved to: St134.pdb
PDB header fixed and saved to: Av-1.pdb
PDB file cleaned and saved to: Av-1.pdb
PDB header fixed and

#**Dataset evaluation**

In [ ]:
import pandas as pd

data = pd.read_csv("BaitTP.csv")
data["Helix"] = ""
data["Sheet"] = ""
data["Coil"] = ""

for i in range(len(data)):
  if data["Acronym"][i] != "CytB":
    pdb_file = f"{data['Acronym'][i]}.pdb"
    print(pdb_file)
    percentages = calculate_secondary_structure_percentage(pdb_file)
    print(percentages)
    data["Helix"][i] = percentages["Helix (%)"]
    data["Sheet"][i] = percentages["Sheet (%)"]
    data["Coil"][i] = percentages["Coil (%)"]
data.to_csv("secondary.csv")


phi29.pdb
{'Helix (%)': 80.18867924528303, 'Sheet (%)': 7.0754716981132075, 'Coil (%)': 12.735849056603774}
GC1.pdb


<ipython-input-148-0263d148f0ab>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["Helix"][i] = percentages["Helix (%)"]
<ipython-input-148-0263d148f0ab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

{'Helix (%)': 7.828282828282829, 'Sheet (%)': 58.333333333333336, 'Coil (%)': 33.83838383838384}
PRD1.pdb


<ipython-input-148-0263d148f0ab>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["Helix"][i] = percentages["Helix (%)"]
<ipython-input-148-0263d148f0ab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

{'Helix (%)': 54.95049504950495, 'Sheet (%)': 22.772277227722775, 'Coil (%)': 22.277227722772277}
Bam35.pdb


...

<ipython-input-148-0263d148f0ab>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["Helix"][i] = percentages["Helix (%)"]
<ipython-input-148-0263d148f0ab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

{'Helix (%)': 83.50515463917526, 'Sheet (%)': 0.0, 'Coil (%)': 16.49484536082474}
LB2.pdb


<ipython-input-148-0263d148f0ab>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["Helix"][i] = percentages["Helix (%)"]
<ipython-input-148-0263d148f0ab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

{'Helix (%)': 90.625, 'Sheet (%)': 0.0, 'Coil (%)': 9.375}


<ipython-input-148-0263d148f0ab>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["Helix"][i] = percentages["Helix (%)"]
<ipython-input-148-0263d148f0ab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [ ]:
data

,Acronym,Score,Nature,Helix,Sheet,Coil
0,phi29,-0.021,TP,80.188679,7.075472,12.735849
1,GC1,-0.012,TP,7.828283,58.333333,33.838384
2,PRD1,-0.028,TP,54.950495,22.772277,22.277228
3,Bam35,-0.031,TP,78.743961,10.628019,10.628019
4,Sato,-0.028,TP,78.636364,10.0,11.363636
...,...,...,...,...,...,...
229,NIT,-0.005,Non-TP,37.5,38.709677,23.790323
230,PbsA,-0.014,Non-TP,87.019231,0.0,12.980769
231,MMA1,-0.016,Non-TP,53.117207,20.947631,25.935162
232,SinA1,-0.009,Non-TP,83.505155,0.0,16.494845
